<h3> Face Detection on Mediapipe</h3>

In [ ]:
# Install mediapipe library #

! pip install mediapipe

In [1]:
import cv2
import mediapipe as mp
from time import time

# Create face detector object: must not be in function to prevent repeated object creation
mp_face_detection = mp.solutions.face_detection
mp_face_detector = mp_face_detection.FaceDetection(min_detection_confidence=0.4)
mp_drawing = mp.solutions.drawing_utils

'''
This function performs face(s) detection on an image using mediapipe deep learning based face detector.
Args:
    image:            The input image with person(s) whose face needs to be detected.
    mp_face_detector: The mediapipe's face detection function required to perform the detection.
    display:          A boolean value that is if set to true the function displays the original input image,
                      and the output image with the bounding boxes, and key points drawn, and also confidence
                      scores, and time taken written and returns nothing.
Returns:
    output_image: A copy of input image with the bounding box and key points drawn and also confidence scores written.
    results:      The output of the face detection process on the input image.
'''
def mpDnnDetectFaces(image, mp_face_detector):


    # Get the height and width of the input image.
    image_height, image_width, _ = image.shape

    # Create a copy of the input image to draw bounding box and key points.
    output_image = image.copy()

    # Convert the image from BGR into RGB format.
    imgRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Perform the face detection on the image.
    results = mp_face_detector.process(imgRGB)

    # Check if the face(s) in the image are found.
    if results.detections:

        # Iterate over the found faces.
        for face_no, face in enumerate(results.detections):
            # Draw the face bounding box and key points on the copy of the input image.
            mp_drawing.draw_detection(image=output_image, detection=face,
                                      keypoint_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0),
                                                                                   thickness=-1,
                                                                                   circle_radius=image_width // 115),
                                      bbox_drawing_spec=mp_drawing.DrawingSpec(color=(0, 255, 0),
                                                                               thickness=image_width // 180))

            # Retrieve the bounding box of the face.
            face_bbox = face.location_data.relative_bounding_box

            x1 = int(face_bbox.xmin * image_width)
            y1 = int(face_bbox.ymin * image_width)

            # Draw a filled rectangle near the bounding box of the face.
            # We are doing it to change the background of the confidence score to make it easily visible
            # cv2.rectangle(output_image, pt1=(x1, y1 - image_width // 20), pt2=(x1 + image_width // 16, y1),
            #               color=(0, 255, 0), thickness=3)

            # Write the confidence score of the face near the bounding box and on the filled rectangle.
            cv2.putText(output_image, text=str(round(face.score[0], 2)), org=(x1, y1 - 25),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=image_width // 600, color=(255, 255, 0),
                        thickness=image_width // 200)

            # keypoints = face.location_data.relative_keypoints
            # print(keypoints)
            # 0: left-eye, 1: right-eye, 2: nose, 3: mouse, 4: left-ear, 5: right-ear

            keypoint_idx = 0
            for keypoint in face.location_data.relative_keypoints:
                cv2.putText(output_image, text=str(keypoint_idx), org=(int(keypoint.x*image_width), int(keypoint.y*image_height)),
                        fontFace=cv2.FONT_HERSHEY_COMPLEX, fontScale=1, color=(0, 255, 255),
                        thickness=2)
                keypoint_idx = keypoint_idx + 1
                # print(keypoint_idx, ": [", int(keypoint.x*image_width), int(keypoint.y*image_height), "] ")


    # Return the output image and results of face detection.
    return output_image, results


start_time = 0
capture = cv2.VideoCapture(0)

while (capture.isOpened()):

    success, frame = capture.read()

    # Check if frame is not read properly then continue to the next iteration to read the next frame.
    if not success:
        continue

    # Flip the frame horizontally for natural (selfie-view) visualization.
    frame = cv2.flip(frame, 1)

    # Get the height and width of the frame.
    frame_height, frame_width, _ = frame.shape

    # Perform face detection using the Mediapipe algorithm.
    frame, _ = mpDnnDetectFaces(frame, mp_face_detector)

    # Write the currently selected method on the frame.
    # cv2.putText(frame, (frame_width // 3, frame_height // 8), cv2.FONT_HERSHEY_PLAIN, 4, (255, 155, 0), 3)

    # Set the time for this frame to the current time.
    end_time = time()

    # Check if the difference between the previous and this frame time &gt; 0 to avoid division by zero.
    if (end_time - start_time) > 0:
        # Calculate the number of frames per second.
        frames_per_second = 1.0 / (end_time - start_time)

        # Write the calculated number of frames per second on the frame.
        cv2.putText(frame, 'FPS: {}'.format(int(frames_per_second)), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)

    # Update the previous frame time to this frame time.
    # As this frame will become previous frame in next iteration.
    start_time = end_time

    cv2.imshow('MediaPipe Face Detection', frame)

    if cv2.waitKey(1) == ord('q'):
        break

capture.release()
cv2.destroyAllWindows()


: 